# 神经网络工具箱Torch.nn
torch.nn其是专门为深度学习而设计的模块。torch.nn的核心数据结构是`Module`，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个**包含很多层的神经网络**。在实际使用中，最常见的做法是继承`nn.Module`，撰写自己的网络/层。

----
## torch.nn实现全连接层
我们已经介绍了`autograd`，`nn`包则依赖于`autograd`包来定义模型并对它们求导。一个`nn.Module`包含各个层和一个`forward(input)`方法，该方法返回`output`。

例如，下面这个神经网络可以对数字进行分类：

![convnet](https://pytorch.org/tutorials/_images/mnist.png)

这是一个简单的前馈神经网络（feed-forward network）。它接受一个输入，然后将它送入下一层，一层接一层的传递，最后给出输出。

下面先来看看如何用nn.Module实现自己的全连接层。输出$\textbf{y}$和输入$\textbf{x}$满足$\textbf{y=Wx+b}$，$\textbf{W}$和$\textbf{b}$是可学习的参数。

In [5]:
import torch as t
from torch import nn #import torch.nn as nn
import torch.nn.functional as F

In [6]:
class Linear(nn.Module): # 继承nn.Module，全连接层类
    def __init__(self, in_features, out_features):# 输入特征，输出特征
        super(Linear, self).__init__() # 等价于nn.Module.__init__(self)
        # super() 函数是用于调用父类(超类)的一个方法
        self.w = nn.Parameter(t.randn(in_features, out_features))
        self.b = nn.Parameter(t.randn(out_features))
    
    def forward(self, x):
        x = x.mm(self.w) # x.@(self.w)
        return x + self.b.expand_as(x)

In [7]:
layer =Linear(4,3) #实例化为对象layer
input=t.randn(2,4)
output=layer(input)
output

tensor([[ 0.5071,  0.0925, -2.7954],
        [ 0.3460,  1.2120,  1.5181]], grad_fn=<AddBackward0>)

可见，全连接层的实现非常简单，其代码量不超过10行，但需注意以下几点：
- 自定义层`Linear`必须继承`nn.Module`，<font color=red>并且在其构造函数中需调用`nn.Module`的构造函数，即`super(Linear, self).__init__()` 或`nn.Module.__init__(self)`，推荐使用第一种用法，尽管第二种写法更直观。</font>
- **在构造函数`__init__`中必须自己定义可学习的参数，并封装成`Parameter`**，如在本例中我们把`w`和`b`封装成`parameter`。`parameter`是一种特殊的`Tensor`，但其默认需要求导（requires_grad = True），感兴趣的读者可以通过`nn.Parameter??`，查看`Parameter`类的源代码。
- `forward`函数实现前向传播过程，其输入可以是一个或多个tensor。
- **无需写反向传播函数，nn.Module能够利用autograd自动实现反向传播，这点比Function简单许多。**
- 使用时，直观上可将layer看成数学概念中的函数，调用layer(input)即可得到input对应的结果。它等价于`layers.__call__(input)`，在`__call__`函数中，主要调用的是 `layer.forward(x)`，另外还对钩子做了一些处理。所以在实际使用中应尽量使用`layer(x)`而不是使用`layer.forward(x)`，关于钩子技术将在下文讲解。
- `Module`中的可学习参数可以通过`named_parameters()`或者`parameters()`返回迭代器，前者会给每个parameter都附上名字，使其更具有辨识度。

可见利用Module实现的全连接层，比利用`Function`实现的更为简单，因其不再需要写反向传播函数。

> 从这里开始，感觉《入门与实践》这本书的讲法很奇怪，讲新的工具箱时不先说参数的作用而是先说代码，对于初学者直接看代码又看不懂，所以打算换apacheCN的[pytorch文档](https://pytorch.apachecn.org/docs/1.2/)和网上大佬改写的pytorch版[《动手学深度学习》](http://tangshusen.me/Dive-into-DL-PyTorch/#/ '《动手学深度学习》')

**一个神经网络的典型训练过程如下：**

- 定义包含一些可学习参数（或者叫权重）的神经网络
- 在输入数据集上迭代
- 通过网络处理输入
- 计算损失（输出和正确答案的距离）
- 将梯度反向传播给网络的参数
- 更新网络的权重，一般使用一个简单的规则：`weight = weight - learning_rate * gradient`
## 定义前馈网络